# Atividade 1
## Item 2 - Problema MAX-QBF - Com linearização

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
def solve_max_qbf_linearized(n, a_coeffs):
    """
    Resolve o problema Unconstrained Binary Quadratic Programming (MAX-QBF)
    utilizando a linearização da função objetivo quadrática.

    Args:
        n (int): Número de variáveis binárias.
        a_coeffs (list): Lista linearizada dos coeficientes da matriz A
                         em formato triangular superior, conforme a Atividade 1 [3].
    """
    # 1. Reconstruir a matriz A (assumindo triangular superior)
    # A matriz A será n x n, com A[i][j] = a_coeffs[k] para j >= i, e 0 caso contrário.
    A = [[0.0] * n for _ in range(n)]
    k = 0
    # Preenche apenas a parte triangular superior da matriz A
    for i in range(n):
        for j in range(i, n): # j começa de i para preencher a diagonal e acima
            A[i][j] = a_coeffs[k]
            k += 1

    # Imprimir a matriz A reconstruída para verificação (útil para pequenas instâncias)
    print("Matriz A de Coeficientes (triangular superior):")
    for row in A:
        print([f"{val:.2f}" for val in row])
    print("-" * 30)

    # 2. Criar um novo modelo Gurobi
    model = gp.Model("MAX_QBF_Linearized_Problem")

    # 3. Adicionar as variáveis binárias originais (x_i)
    # x[i] representa a i-ésima variável binária (indexado de 0 a n-1)
    x = model.addVars(n, vtype=GRB.BINARY, name="x")

    # 4. Adicionar as variáveis de linearização (y_ij)
    # y[i, j] representa o produto x_i * x_j.
    # Como A é triangular superior [3], só precisamos de y_ij para i <= j.
    # As variáveis são indexadas de 0 a n-1.
    y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

    # 5. Definir a função objetivo linearizada
    # A função objetivo original: Z = ∑_{i=0 to n-1} ∑_{j=0 to n-1} a_ij * x_i * x_j
    # Com a linearização, e considerando que A[i][j] é 0 para i > j,
    # torna-se: Z = ∑_{i=0 to n-1} ∑_{j=i to n-1} a_ij * y_ij
    obj = gp.quicksum(A[i][j] * y[i, j] for i in range(n) for j in range(n) if i <= j)
    
    model.setObjective(obj, GRB.MAXIMIZE)

    # 6. Adicionar as restrições de linearização
    # Para cada par (i, j) onde y_ij é definido (i <= j), adicionamos as três restrições.
    for i in range(n):
        for j in range(i, n): # Iterar sobre i <= j
            # Restrição 1: y_ij <= x_i
            model.addConstr(y[i, j] <= x[i], f"linear_y{i}_{j}_le_x{i}")
            # Restrição 2: y_ij <= x_j
            model.addConstr(y[i, j] <= x[j], f"linear_y{i}_{j}_le_x{j}")
            # Restrição 3: y_ij >= x[i] + x[j] - 1
            model.addConstr(y[i, j] >= x[i] + x[j] - 1, f"linear_y{i}_{j}_ge_sum_minus_1")
            
            # Note: Para i == j, as restrições forçam y[i,i] = x[i] corretamente.

    # Configurar limite de tempo (10 minutos conforme a Atividade 1, seção 4.2) [4]
    model.Params.TimeLimit = 600 # segundos

    # 7. Otimizar o modelo
    print("Iniciando otimização do MAX-QBF (Modelo Linearizado)...")
    model.optimize()

    # 8. Obter e exibir os resultados
    print("\n" + "=" * 40)
    print("Resultados da Otimização (Modelo Linearizado):")
    print("=" * 40)

    if model.status == GRB.OPTIMAL:
        print(f"Status: Ótimo (Optimal)")
        print(f"Valor da Solução Ótima (Z): {model.ObjVal:.4f}")
        print("Valores das Variáveis x:")
        for i in range(n):
            print(f"  x[{i+1}] = {int(x[i].X)}") # Apresentando como x1 a xn
        
        # Opcional: Imprimir os valores de y_ij para verificar a linearização
        # print("\nValores das Variáveis y (x_i * x_j):")
        # for i in range(n):
        #     for j in range(i, n):
        #         if y[i,j].X > 0.5: # Verifica com tolerância para ponto flutuante
        #             print(f"  y[{i+1},{j+1}] = {int(y[i,j].X)} (x{i+1}={int(x[i].X)}, x{j+1}={int(x[j].X)})")

    elif model.status == GRB.TIME_LIMIT:
        print(f"Status: Limite de tempo atingido (Time Limit)")
        print(f"Melhor Solução Encontrada (Z): {model.ObjVal:.4f}")
        # Para problemas com limite de tempo, é importante reportar o gap de otimalidade [4]
        print(f"Gap de Otimalidade: {model.MIPGap * 100:.2f}%") 
        print("Valores das Variáveis x na melhor solução encontrada:")
        for i in range(n):
            print(f"  x[{i+1}] = {int(x[i].X)}")
    elif model.status == GRB.INFEASIBLE:
        print("Status: Inviável (Infeasible)")
    else:
        print(f"Status da Otimização: {model.status}")

In [3]:
# Dados de exemplo para n=5 da Atividade 1
n_example = 5
a_coeffs_example = [
    3, 1, -2, 0, 3,  # a11 a12 a13 a14 a15
      -1, 2, 1, -1,  # a22 a23 a24 a25
          2, -2, 4,  # a33 a34 a35
              0, 5,  # a44 a45
                 3   # a55
    ]
    
print("--- Executando com o exemplo n=5 (linearizado) ---")
solve_max_qbf_linearized(n_example, a_coeffs_example)

--- Executando com o exemplo n=5 (linearizado) ---
Matriz A de Coeficientes (triangular superior):
['3.00', '1.00', '-2.00', '0.00', '3.00']
['0.00', '-1.00', '2.00', '1.00', '-1.00']
['0.00', '0.00', '2.00', '-2.00', '4.00']
['0.00', '0.00', '0.00', '0.00', '5.00']
['0.00', '0.00', '0.00', '0.00', '3.00']
------------------------------
Set parameter Username
Set parameter LicenseID to value 2696041
Set parameter TimeLimit to value 600
Iniciando otimização do MAX-QBF (Modelo Linearizado)...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 45 rows, 30 columns and 100 nonzeros
Model fingerprint: 0xa5e91f41
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range 

In [6]:
# Exemplo com n=3 para ilustrar mais claramente a matriz A e o objetivo
print("\n" + "=" * 40)
print("Exemplo Adicional: n=3")
print("=" * 40)
n_small = 3

a_coeffs_small = [-5, -6, 7, 8, 9, -10] 
solve_max_qbf_linearized(n_small, a_coeffs_small)


Exemplo Adicional: n=3
Matriz A de Coeficientes (triangular superior):
['-5.00', '-6.00', '7.00']
['0.00', '8.00', '9.00']
['0.00', '0.00', '-10.00']
------------------------------
Set parameter TimeLimit to value 600
Iniciando otimização do MAX-QBF (Modelo Linearizado)...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 18 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x45d529ad
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 3.0000000
Presolve removed 18 rows and 12 columns
Preso